<center><h1>Modeling an Electron in a Field of Secured Point Charges</h1></center>

<h2>Introduction</h2>
<p>Kinetics typically makes sense. You can see kinetics. But, you can't really see an electric field. The idea that an electron will move towards a positive charge and away from a negative charge might make sense, but it can be difficult to visualize where an electron might go in a more complicated field of charges. This project is about helping students in physics classes understand how electrons are moved in electric fields</p>
<p>In my project, we are looking at an idealized simulation. The only forces present on the electron are the forces created by the charges, and those charges are all point charges fixed in place that won't be affected by each other. While not an extrememly realistic situation, it is a good situation for beginners to get a better understanding.</p>
<p>The simulation relies on Coulomb's Law, which is used to calculate the force on a point charge by another point charge. It also uses Newton's Second Law (f=ma) to get acceleration from force. Finally, we use Reimann sums, as velocity is the area under the curve of acceleration and position the area under the curve of velocity.</p>

<h2>Methodology</h2>
<h3>Imports and Libraries</h3>

In [2]:
import math #basic python math library, used for square rooting and getting pi
#https://docs.python.org/3/library/math.html#
import numpy as np #common library, source of numpy arrays which are easier to manipulate as vectors than your average python list
#https://numpy.org/doc/stable/index.html

from scipy.constants import e, m_e, epsilon_0 
#physics constants I need, the charge of a proton or electron (e), the mass of an electron (m_e), and epsilon naught
#https://docs.scipy.org/doc/scipy/reference/constants.html

import matplotlib.pyplot as plt #for plotting data
#https://matplotlib.org/3.5.3/api/_as_gen/matplotlib.pyplot.html
import matplotlib.animation as animation #to create animation
#https://matplotlib.org/stable/api/animation_api.html
from IPython.display import HTML #making it possible to display my animation
#https://ipython.readthedocs.io/en/stable/api/generated/IPython.display.html

<p>The first part of making the simulation is getting all of the necessary functions and constants from libraries.</p>

<h4>Level 1</h4>
<p>Using libraries in code is like using an actual library, where you borrow parts of other people's code to solve a problem the way you would borrow a book from a library. For this simulation, we need to borrow from a couple of different libraries.</p> 
<p>"math" is a built-in library to Python, it is like a school library, where you still need to borrow books but you don't have to go anywhere to get them. "math" contains sqrt(), which allows us to square root things, and pi, which is the numner pi.</p>
<p>"numpy" is a library that is all about a special type of list, known as a ndarray. ndarrays are useful because they can work as vectors, allowing us to get new velocities and positions through addition.</p>
<p>"scipy.constants" is a library that contains various scientific constants. The constants we need are "e", which is the charge of a single proton or electron, "m_e", which is the mass of an electron, and "epsilon_0", which is the electric constant when in a vacuum, which we need for Coulomb's Law.</p>
<p>from "matplotlib" we take "pyplot" and "animation". "pyplot" allows us to make plots (specifically scatter plots for our purposes), while "animation" will let us turn those plots into animations.</p>
<p>Lastly, from "IPython.display" we have "HTML", which allows us to make the animation visible to users, not just coders.</p>

<h4>Level 2</h4>
<p>Here, we are importing a couple of full libraries (math, numpy), parts of libraries (such as matplotlib.pyplot and .animation), and specific functions and constants (e, m_e, etc)</p>
<p>"math" and "numpy" are allowing us to do the arithmetic we need to do to create the simulation. We need "math" because Coulomb's law has pi, and we need to take a square root to get the distance between two points, as the distance is the magnitude of the vector between them. "numpy" gives us ndarrays, which are a special type of list that have a fixed length, but you can do mathematical operation on them, such as adding them together. A normal Python list just makes a new list containing both lists, while if you add two ndarrays of the same size, they add the elements that are in the same spot to create an ndarray of the same length as them, which will allow us to add the change in position to the position to get the new position</p>
<p>Next, we're getting parts of libraries, "matplotlib.pyplot" and "matplotlib.animation". These are the backbone of the simulation, as while the other parts work to do the science behind the simulation, these make the simulation actually exist as visuals instead of strings of numbers. "matplotlib.pyplot" is where the scatter plot of charges and electron are being made, along with a legend to help make the simulation more understandable. Then, "matplotlib.animation" allows us to have the scatter plot update with the movements of the electron, turning the plot into an animation.</p>
<p>"scipy" has constants and functions for scientific computing, but this project only requies constants. Since we just need a couple of specific constants, I found the names for those constants in "scipy.constants" and imported them. "IPython.display" has various functions for displaying code, the one we use it "HTML", which takes in data then creates an object that can be viewed on the frontend. By using this, we are able to fully see the animation and control whether it is going or not, since if we just show the plot without moving it into the frontend we can only see one frame.</p>

<h4>Level 3</h4>
<p>There isn't more to say here, import statements aren't extremely complicated.</p>

In [3]:
class Charge: #class to create Charge objects, which will make it easier to work with charges

    def __init__(self, x, y, charge): #initiation function, magic method
        self.position = np.array([float(x), float(y)]) ##creating the position, using the given x and y coordinates, as an array
        #https://numpy.org/doc/stable/reference/generated/numpy.array.html
        self.charge = charge #getting the charge of the particle

    def get_position(self): #function to get the position of a charge
        return self.position #return the position

    def get_charge(self): #function to get the charge of a charge
        return self.charge #return the charge

<p>The second thing for this simulation is making it possible to create charges through designing a Charge Object.</p>

<h4>Level 1</h4>
<p>In Python, a class is how you make an Object. Programming Objects are containers for when you want to make specific types of things, or want to have one variable that is containing multiple elements of information. Objects can have variable and functions, called methods, held inside of them. Then, when you create an Object, it will have those variables and be able to use those methods.</p>
<p>Here is a Charge Objects. Charges have a two pieces of information that they need to exist, a location and a charge. So, when a charge is created, you need to enter in the x and y positions, so it has a location, and the value (negative or positive) of the charge. Alongside creating the charge, we have two methods, get_position() and get_charge(). The purpose of these is so when we want to know the position of the charge or the charge of the charge in the future, we can easily access it through these getter methods.</p>

<h4>Level 2</h4>
<p>The purpose of the Charge Object is to have a consistent system for charges, as charges as held stationary in this model, there is nothing that can change them, but they make it much easier to store the data. The position of a charge is held as a ndarray instead of a list because when you need to find the distance (in vector form) between a charge and an electron, it is much easier to use an ndarray for both because then you can just subtractthe position of a charge from the position of the electron. While I'm not making these variable private, I'm still using getter methods (get_position() and get_charge()) to make the code more clear and because in languages other than Python require these and it's a good habit to keep.</p>

<h4>Level 3</h4>
<p>This is an Object that I'll be using to hold the charges. I've made the decision to use getter methods as practice for if I'm not in Python, though technically this Object would work with just initiation. The initiation is a magic method. If a user does not give a float for a position, the position will be transformed into a float, which makes math easier, but it will cause problems if x and y aren't numbers. While this problem is not dealt with in the Object itself, part of the process to create the charges will make it so users have to give a valid number for the x and y values, along with the charge value.</p>

In [4]:
class Electron: #class to create Electron objects, which will allow me to make functions for electrons

    def __init__(self, x, y): #initiation function, magic method
        self.position = np.array([float(x), float(y)]) #creating the position, using the given x and y coordinates, as an array
        self.charge = -e #all electrons have a charge of -e
        self.velocity = np.array([0.0, 0.0]) #start with 0 velocity
        self.acceleration = np.array([0.0, 0.0]) #start with 0 acceleration

    def get_position(self): #function to get the position of an electrno
        return self.position #return the position

    def get_velocity(self):
        return self.velocity
    
    def get_acceleration(self): #return the acceleration
        return self.acceleration #return acceleration

    def get_force(self, charges): #function to get the sum of all the forces on an electron
        force = np.array([0.0, 0.0]) #start with 0
        for charge in charges: #then go through each charge
            r = math.sqrt((charge.get_position() - self.get_position())[0] ** 2 + (charge.get_position() - self.get_position())[1] ** 2)
            #get the distance from electron to charge
            force += (self.get_position() - charge.get_position()) * charge.get_charge() * self.charge / 4 / math.pi / epsilon_0 / (r ** 3)
            #use Coulomb's Law to find the force a single charge places on the electron and add it to the current force
        self.acceleration = force / m_e / 100 #Newton's second law, f=ma, so a=f/m

    def update_velocity(self): #changing velocity
        self.velocity += self.acceleration #small increments, velocity changes by acceleration

    def update_position(self): #changing position
        self.position += self.velocity #small increments, position changes by velocity

<p>Next is creating an Electron Object, that allows the electron to hold different values (position, velocity, acceleration, and charge) and gives me the ability to give Electron's methods for the various things that happen to electrons.</p>

<h4>Level 1</h4>

<h4>Level 2</h4>
<p>The Electron Object is similiar to the Charge Object, but it has more capabilties because we need to do more with it. In addition to having a position and charge, electrons have velocity and acceleration because they move. Also, since you never need to get the charge of an electron, we don't have a getter method for charge but we do for acceleration, velocity, and position. Additionally, we have methods that impact of our electron.</p>
<p>The first method is get_force(self, charges), which takes all of the charges present in our field. It computes the amount of force each individual charge places on the electron, then adds them all together to get the final force. Force is found using Coulomb's Law, which computes the force on a point charge created by another point charge. Couloumb' Law is 
$$
use by the mass of an electron. 
$$
We use a loop to go through each charge, finding the distance between the charge and the electron, and plugging all of our variable in. Once we have the complete charge, we use Newton's Second Law (f=ma) to determine the acceleration as force divided by the mass of an electron. I divide this number by 100 in order to m make a better model, as it will otherwise overshoot because the model corrects every frame, not immediately, which can result in problems when the electron moves a lot at once.</p>
<p>update_velocity() and update_position() both opperate by adding the acceleration and velocity respectively to the current velocity and position. Since velocity is the area under the curve of acceleration, you can compute velocity by adding together all of the accelerations, and since we move a little at a time, we can use the method of adding the current acceleration to the current velocity to get the new velocity. This same method works for position, as position is the area under the curve of velocity.</p>

<h4>Level 3</h4>

In [5]:
def electron_update(charges, electron): #function to update the state of the electron

    electron.get_force(charges) #getting the current force
    electron.update_velocity() #updating velocity
    electron.update_position() #updating position

<p>Here is a function that allows us to go through all of the elements of getting the movement of an electron.</p>

<h4>Level 1</h4>

<h4>Level 2</h4>

<h4>Level 3</h4>

In [1]:
def gen(charges, electron): #making a generator function!
    i = 1 #letting us start with 1 frame
    sep = True #boolean to determine if you should continue
    while sep:
        #print(i)
        for charge in charges: #look at each charge
            if abs(electron.get_position()[0] - charge.get_position()[0]) < 2 and abs(electron.get_position()[1] - charge.get_position()[1]) < 2:
            #if the electron is practically on the charge
                sep = False #don't continue
                print("on charge") #let us know we are on a charge
                print(charge.get_position(), electron.get_position()) #print both locations
                print(abs(electron.get_position()[0] - charge.get_position()[0]), abs(electron.get_position()[1] - charge.get_position()[1])) #print distance
                break #exit this loop
        if abs(electron.get_position()[0]) > 100 or abs(electron.get_position()[1]) > 100: #if it's outside of the boundaries of the simulation
            print("left boundaries")
            sep = False #don't continue
        if np.array_equal(electron.get_velocity(), [0.0,0.0]) and np.array_equal(electron.get_acceleration(), [0.0, 0.0]): #if the velocity + acceleration is 0
            print("no velocity/acceleration") #let us know that the velocity/acceleration is 0
            print(electron.get_velocity(), electron.get_acceleration()) #prove it
            sep = False #don't continue
        if sep == True: #if we still should continue
            i += 1 #up the frame number
            #print(i)
            yield i #iteration version of return

<p>This is a generator functions, which allows for controlling how many frames the animation runs for without having to specify a number ahead of time.</p>

<h4>Level 1</h4>

<h4>Level 2</h4>

<h4>Level 3</h4>

In [7]:
def create_animation(charges, electron): #creating the animation

    fig, ax = plt.subplots() #needed elements of the plot
    #https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.subplots.html
    
    scatter = ax.scatter([c.get_position()[0] for c in charges if c.get_charge() > 0], [c.get_position()[1] for c in charges if c.get_charge() > 0], c="r", label="+ charges")
    #placing all of the positive charges on the plot, coloring them red
    #https://matplotlib.org/stable/api/_as_gen/matplotlib.axes.Axes.scatter.html#matplotlib.axes.Axes.scatter
    scatter = ax.scatter([c.get_position()[0] for c in charges if c.get_charge() < 0], [c.get_position()[1] for c in charges if c.get_charge() < 0 ], c="b", label="- charges")
    #placing all of the negative charges on the plot, coloring them blue
    scatter = ax.scatter(electron.get_position()[0], electron.get_position()[1], c="g", label="electron")
    #place the electron on the plot, coloring it green

    ax.set_xlim([-100, 100]) #making the x-axis -100 to 100
    #https://matplotlib.org/stable/api/_as_gen/matplotlib.axes.Axes.set_xlim.html
    ax.set_ylim([-100, 100]) #making the y-axis -100 to 100
    #https://matplotlib.org/stable/api/_as_gen/matplotlib.axes.Axes.set_ylim.html
    ax.legend() #show the legend
    #https://matplotlib.org/stable/api/_as_gen/matplotlib.axes.Axes.legend.html
    
    def update(frame): #update function!

        electron_update(charges, electron) #call the update function to set the values
        scatter.set_offsets(electron.get_position()) #update the position of the electron
        #https://matplotlib.org/stable/api/collections_api.html#matplotlib.collections.AsteriskPolygonCollection.set_offsets
        return scatter #return the new scatter plot

    electron.get_force(charges) #not have starting acceleration 0
    anim = animation.FuncAnimation(fig=fig, func=update, frames=gen(charges, electron), interval=10, save_count=1000) #create the animation
    #https://matplotlib.org/stable/api/_as_gen/matplotlib.animation.FuncAnimation.html
    plt.close() #close the plot

    return HTML(anim.to_jshtml()) #return the HTML to make the animation
    #https://matplotlib.org/stable/api/_as_gen/matplotlib.animation.Animation.html#matplotlib.animation.Animation.to_jshtml

<p>This section is the meat of the program, as it is where the animation is actually being created.</p>

<h4>Level 1</h4>

<h4>Level 2</h4>

<h4>Level 3</h4>

In [8]:
print("This program will create a 2d simulation with point charges fixed in place and an unsecured electron.")
print("The dimensions are (-100, 100) by (-100, 100).")
print()
print("First, create your Charges.")
q = int(input("Number of Charges (int): "))
charges = []
for i in range(q):
    x = float(input("Enter x coordinate (float): "))
    y = float(input("Enter y coordinate (float): "))
    c = float(input("Enter charge (float): "))
    charges.append(Charge(x, y, c * e))
print("Second, locate your Electron.")
x = float(input("Enter x coordinate (float): "))
y = float(input("Enter y coordinate (float): "))
electron = Electron(x, y)
create_animation(charges, electron)

This program will create a 2d simulation with point charges fixed in place and an unsecured electron.
The dimensions are (-100, 100) by (-100, 100).

First, create your Charges.


Number of Charges (int):  0


Second, locate your Electron.


Enter x coordinate (float):  0
Enter y coordinate (float):  0


no velocity/acceleration
[0. 0.] [0. 0.]


C:\Users\dbart\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\matplotlib\animation.py:1741: UserWarning: Can not start iterating the frames for the initial draw. This can be caused by passing in a 0 length sequence for *frames*.

If you passed *frames* as a generator it may be exhausted due to a previous display or save.
  warnings.warn(


<p>Where the program is actually running.</p>

<h4>Level 1</h4>

<h4>Level 2</h4>

<h4>Level 3</h4>